In [ ]:
import numpy as np

np.set_printoptions(precision=4, suppress=True)
print("numpy", np.__version__)

## 1) Shapes and dtypes

Two frequent sources of bugs:

- silently getting shape `(n,)` vs `(n,1)` vs `(1,n)`
- silently getting integer dtype when you expected floats.


In [ ]:
x = np.array([1, 2, 3])
y = np.array([[1], [2], [3]])
print("x.shape", x.shape)
print("y.shape", y.shape)

a = np.array([1, 2, 3], dtype=np.int32)
b = a / 2
print("a dtype", a.dtype, "b dtype", b.dtype, "b", b)

## 2) Views vs copies

Slicing usually returns a **view** (shares memory). Advanced indexing usually returns a **copy**.

This matters for correctness and performance.


In [ ]:
m = np.arange(12).reshape(3, 4)
v = m[:, :2]
v[0, 0] = -999
print("m after view write:", m)

m2 = np.arange(12).reshape(3, 4)
c = m2[[0, 2], :]  # advanced indexing copy
c[0, 0] = -999
print("m2 unchanged by c write:", m2)

## 3) Broadcasting (the rule you must internalize)

Two shapes are compatible if, from the trailing dimensions, each pair is equal or one of them is 1.

Broadcasting is _conceptual_ repetition without materializing data.


In [ ]:
A = np.random.default_rng(0).normal(size=(4, 3))
mu = A.mean(axis=0)  # (3,)
centered = A - mu  # broadcasts (4,3) - (3,)
print("mu shape", mu.shape)
print("centered shape", centered.shape)

mu_col = mu.reshape(1, -1)  # (1,3) also broadcasts
print(np.allclose(centered, A - mu_col))

## 4) Vectorization patterns: distances and similarities

Example: compute pairwise squared Euclidean distances between two sets of vectors.

If `X` is `(n,d)` and `Y` is `(m,d)`, we want an `(n,m)` matrix `D` where:

$$D_{ij} = X_i - Y_j_2^2$$

We’ll do it without explicit loops.


In [ ]:
rng = np.random.default_rng(1)
X = rng.normal(size=(5, 3))
Y = rng.normal(size=(7, 3))

# (n,1,d) - (1,m,d) -> (n,m,d), then sum over d
D = ((X[:, None, :] - Y[None, :, :]) ** 2).sum(axis=2)
print(D.shape)
print(D[:2, :3])

### Alternative via dot products

Use: $x-y^2 = x^2 + y^2 - 2x^Ty$

This is often faster for large problems.


In [ ]:
X2 = (X**2).sum(axis=1)[:, None]  # (n,1)
Y2 = (Y**2).sum(axis=1)[None, :]  # (1,m)
XY = X @ Y.T  # (n,m)
D2 = X2 + Y2 - 2 * XY
print(np.max(np.abs(D - D2)))

## 5) `einsum`: express linear algebra cleanly

`einsum` can make tensor contractions explicit and readable once you get used to the notation.


In [ ]:
# Batch dot products: for arrays (n,d) and (n,d) -> (n,)
u = rng.normal(size=(4, 3))
v = rng.normal(size=(4, 3))
dots = np.einsum("nd,nd->n", u, v)
print(dots)
print(np.allclose(dots, np.sum(u * v, axis=1)))

## 6) Numerical stability: softmax and log-sum-exp

Softmax is a classic ‘looks simple, breaks in practice’ function.

Naive: `exp(x) / sum(exp(x))` overflows for large `x`.

Stable: subtract max before exponentiating.


In [ ]:
def softmax_stable(z: np.ndarray, axis: int = -1) -> np.ndarray:
    z = np.asarray(z)
    zmax = np.max(z, axis=axis, keepdims=True)
    exp = np.exp(z - zmax)
    return exp / np.sum(exp, axis=axis, keepdims=True)


z = np.array([1000.0, 1001.0, 999.0])
print(softmax_stable(z))
print("sum:", softmax_stable(z).sum())

# Exercises

## Exercise A — Batch cosine similarity (no loops)

Given `X` shape `(n,d)` and `Y` shape `(m,d)`, compute cosine similarity matrix `(n,m)`.

## Exercise B — PCA via SVD

Implement PCA: center data, compute SVD, project to top-k components.

## Exercise C — Verify broadcasting intuition

Create 3 arrays with different shapes and predict the result shape before executing the operation.


In [ ]:
# Starter for Exercise A
def cosine_sim_matrix(X: np.ndarray, Y: np.ndarray, eps: float = 1e-12) -> np.ndarray:
    X = np.asarray(X, dtype=np.float64)
    Y = np.asarray(Y, dtype=np.float64)
    Xn = np.linalg.norm(X, axis=1, keepdims=True)
    Yn = np.linalg.norm(Y, axis=1, keepdims=True)
    # (n,d) @ (d,m) -> (n,m)
    return (X @ Y.T) / ((Xn + eps) * (Yn.T + eps))


X = rng.normal(size=(3, 4))
Y = rng.normal(size=(5, 4))
S = cosine_sim_matrix(X, Y)
print(S.shape)
print(S[:2, :3])